In [6]:
import os, glob
import pandas as pd
from datetime import datetime
import numpy as np
import rasterio
import xgboost as xgb

In [7]:
def get_index_and_mask_train(fp_mask, nodata_value=0):
    src = rasterio.open(fp_mask)
    mask = src.read()[0].flatten()
    index_nodata = np.where(mask == nodata_value)
    mask_train = np.delete(mask, index_nodata)
    return mask_train, index_nodata

In [8]:
def get_df_flatten_train(fp_img, list_number_band, index_nodata, name_atrr):
    src = rasterio.open(fp_img)
    # return to img train
    list_band_have = list(range(1,src.count+1))
    dfObj = pd.DataFrame()
    if set(list_number_band).issubset(list_band_have):
        img = src.read(list_number_band)
        i = 0
        for band in img:
            band = band.flatten()
            band = np.delete(band, index_nodata)
            name_band = f"band {list_number_band[i]}_{name_atrr}"
            dfObj[name_band] = band
            i+=1
        return dfObj
    else:
        miss = np.setdiff1d(list_number_band, list_band_have)
        print("*"*15, "ERROR", "*"*15)
        print(f"Image dont have band : {miss.tolist()}")

In [9]:
def get_list_image_by_time(dir_img):
    list_name_file = os.listdir(dir_img)
    print(list_name_file)
    list_time = []
    for name in list_name_file:
        list_time.append(name[17:25])
    list_time.sort(key=lambda date: datetime.strptime(date, '%Y%m%d'))
    return list_time[:9]

In [10]:
def create_csv_train(list_fp_img, list_name_file_sort, list_number_band, index_nodata,fp_csv):
    fp_img_first = [s for s in list_fp_img if list_name_file_sort[0] in s][0]
    df = get_df_flatten_train(fp_img_first, list_number_band, index_nodata, list_name_file_sort[0])
    print(df.shape,"a")
    for name_file_sort in list_name_file_sort[1:]:
        fp_img = [s for s in list_fp_img if name_file_sort in s][0]
        df1 = get_df_flatten_train(fp_img, list_number_band, index_nodata, name_file_sort)
        df = pd.concat([df, df1], axis=1)
        print(df.shape)
    df.to_csv(fp_csv)       

In [11]:
def chao_ngay_df(df, list_colums):
    df_get = pd.concat([df.pop(x) for x in list_colums], axis=1)
    return pd.concat([df,df_get ], axis=1)

In [12]:
def make_time_seris(fp_csv):
    datasets = pd.read_csv(fp_csv)
    list_name_band = datasets.columns.to_list()
    tmp_df = datasets.copy()
    for i in range(8):
        list_7_band = list_name_band[0:7]
        tmp_df = chao_ngay_df(tmp_df, list_7_band)
        tmp_df.columns = list_name_band
        datasets = pd.concat([datasets, tmp_df])
    print(datasets.shape)
    return datasets


In [9]:
def create_data_train_Xgboost(df, training_per = 0.8):
    datasets = df.iloc[:, 2:]
    print(datasets)
    ds_train = datasets.sample(frac=training_per)
    ds_test = datasets[~datasets.isin(ds_train)].dropna()

    X_train = ds_train.iloc[:, :-1]
    Y_train = ds_train.iloc[:, -1]
    X_test = ds_test.iloc[:, :-1]
    Y_test = ds_test.iloc[:, -1]

    return X_train, Y_train, X_test, Y_test

In [10]:
dir_img = r"E:\WORK\Mongodia\Data_cut_img\Img"
fp_mask = r"E:\WORK\Mongodia\Data_cut_img\label_shp\mask.tif"
list_number_band = [1,2,3,4,5,6,7]
out_fp_csv_train = r"E:\WORK\Mongodia\Data_cut_img\train.csv"

list_name_file_sort = get_list_image_by_time(dir_img)
list_fp_img = glob.glob(os.path.join(dir_img, "*.tif"))

mask_train, index_nodata = get_index_and_mask_train(fp_mask, nodata_value=0)
if not os.path.exists(out_fp_csv_train):
    create_csv_train(list_fp_img, list_name_file_sort, list_number_band, index_nodata,out_fp_csv_train)

df = make_time_seris(out_fp_csv_train)
mask_train = np.tile(mask_train, 9) - 1
print(np.unique(mask_train))
df['label'] = mask_train
# df.to_csv('a.csv')
# df = df.iloc[:, 1:]
df = df.reset_index()

['LC08_L2SP_133026_20210201_20210302_02_T1_0.tif', 'LC08_L2SP_133026_20210305_20210312_02_T1_0.tif', 'LC08_L2SP_133026_20210406_20210415_02_T1_0.tif', 'LC08_L2SP_133026_20210609_20210615_02_T1_0.tif', 'LC08_L2SP_133026_20210913_20210924_02_T1_0.tif', 'LC08_L2SP_133026_20211031_20211109_02_T1_0.tif', 'LC08_L2SP_133026_20220119_20220127_02_T1_0.tif', 'LC08_L2SP_133026_20220204_20220212_02_T1_0.tif', 'LC09_L2SP_133026_20211121_20220120_02_T1_0.tif', 'LC09_L2SP_133026_20211226_20220121_02_T1_0.tif', 'LC09_L2SP_133026_20220127_20220129_02_T1_0.tif', 'LC09_L2SP_133026_20220212_20220214_02_T1_0.tif', 'LC09_L2SP_133026_20220228_20220302_02_T1_0.tif']
(98559, 64)
[0 1 2 3 4 5]


## Training

In [324]:
# df = df.loc[~df.index.duplicated(), :]

In [325]:
fp_model_save = r"E:\WORK\Mongodia\Data_cut_img\mode_num_round_100_max_depth7_gpu.model"

X_train, Y_train, X_test, Y_test = create_data_train_Xgboost(df, training_per = 0.8)
print(X_test)
print('Training ...')
dtrain = xgb.DMatrix(X_train, label=Y_train)
dtest = xgb.DMatrix(X_test, label=Y_test)
num_round = 100
param = {'max_depth': 7, 'eta': 1, 'objective': 'multi:softmax'}
param['nthread'] = 5
param['eval_metric'] = 'auc'
param['num_class'] = 6
param['gpu_id'] = 0
evallist = [(dtest, 'eval'), (dtrain, 'train')]
bst = xgb.train(param, dtrain, num_round, evallist)
bst.save_model(fp_model_save)

       band 1_20210201  band 2_20210201  band 3_20210201  band 4_20210201  \
0                41543            42574            41006            41195   
1                41799            42997            41175            41326   
2                41253            42181            40377            40714   
3                41045            41826            40260            40545   
4                41686            42655            40912            41313   
...                ...              ...              ...              ...   
98554             9959             8520             7378             4963   
98555             9458             7211             6241             3410   
98556             8367             6370             5784             2740   
98557             8265             6305             5905             2673   
98558             7919             6221             6083             3370   

       band 5_20210201  band 6_20210201  band 7_20210201  band 1_20210305  

## Predict 


In [13]:
def get_df_flatten_predict(fp_img, list_number_band, name_atrr):
    src = rasterio.open(fp_img)
    # return to img train
    list_band_have = list(range(1,src.count+1))
    dfObj = pd.DataFrame()
    if set(list_number_band).issubset(list_band_have):
        img = src.read(list_number_band)
        i = 0
        for band in img:
            band = band.flatten()
            name_band = f"band {list_number_band[i]}_{name_atrr}"
            dfObj[name_band] = band
            i+=1
        return dfObj
    else:
        miss = np.setdiff1d(list_number_band, list_band_have)
        print("*"*15, "ERROR", "*"*15)
        print(f"Image dont have band : {miss.tolist()}")


In [14]:
def create_predict_df(list_fp_img, list_name_file_sort, list_number_band):
    fp_img_first = [s for s in list_fp_img if list_name_file_sort[0] in s][0]
    df = get_df_flatten_predict(fp_img_first, list_number_band, list_name_file_sort[0])
    print(df.shape,"a")
    for name_file_sort in list_name_file_sort[1:]:
        fp_img = [s for s in list_fp_img if name_file_sort in s][0]
        df1 = get_df_flatten_predict(fp_img, list_number_band, name_file_sort)
        df = pd.concat([df, df1], axis=1)
        print(df.shape)
    return df

In [15]:
def create_predict_df_v2(list_fp_img, list_name_file_sort, list_number_band, csv_file):
    fp_img_first = [s for s in list_fp_img if list_name_file_sort[0] in s][0]
    df = get_df_flatten_predict(fp_img_first, list_number_band, list_name_file_sort[0])
    print(df.shape,"a")
    df.to_csv(csv_file)
    del df
    for name_file_sort in list_name_file_sort:
        fp_img = [s for s in list_fp_img if name_file_sort in s][0]
        df1 = get_df_flatten_predict(fp_img, list_number_band, name_file_sort)
        df = pd.read_csv(csv_file)
        df = pd.concat([df, df1], axis=1)
        df.to_csv(csv_file)
        print(df.shape)
    # return df

In [16]:
dir_img = r"E:\WORK\Mongodia\Data_cut_img\Img"
fp_model_save = r"E:\WORK\Mongodia\Data_cut_img\mode_num_round_100_max_depth7_gpu.model"
list_number_band = [1,2,3,4,5,6,7]

list_name_file_sort = get_list_image_by_time(dir_img)
list_fp_img = glob.glob(os.path.join(dir_img, "*.tif"))
df_data = create_predict_df(list_fp_img, list_name_file_sort, list_number_band)


model = xgb.Booster({'nthread': 16})
model.load_model(fp_model_save)
# model.set_param({"predictor": "gpu_predictor"})
df_data.shape

df_data = xgb.DMatrix(df_data)
X_predict = model.predict(df_data, pred_contribs=True)
X_predict = X_predict.astype('uint16')

['LC08_L2SP_133026_20210201_20210302_02_T1_0.tif', 'LC08_L2SP_133026_20210305_20210312_02_T1_0.tif', 'LC08_L2SP_133026_20210406_20210415_02_T1_0.tif', 'LC08_L2SP_133026_20210609_20210615_02_T1_0.tif', 'LC08_L2SP_133026_20210913_20210924_02_T1_0.tif', 'LC08_L2SP_133026_20211031_20211109_02_T1_0.tif', 'LC08_L2SP_133026_20220119_20220127_02_T1_0.tif', 'LC08_L2SP_133026_20220204_20220212_02_T1_0.tif', 'LC09_L2SP_133026_20211121_20220120_02_T1_0.tif', 'LC09_L2SP_133026_20211226_20220121_02_T1_0.tif', 'LC09_L2SP_133026_20220127_20220129_02_T1_0.tif', 'LC09_L2SP_133026_20220212_20220214_02_T1_0.tif', 'LC09_L2SP_133026_20220228_20220302_02_T1_0.tif']
(1199042, 7) a
(1199042, 14)
(1199042, 21)
(1199042, 28)
(1199042, 35)
(1199042, 42)
(1199042, 49)
(1199042, 56)
(1199042, 63)


## Mạng Dense

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  7 16:15:55 2022

@author: SkyMap
"""

# from email.mime import base
# from osgeo import gdal
import numpy as np
import pandas as pd
# import geopandas as gp
import os, glob

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, ReLU
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# from keras.models import load_model
    
import rasterio
from sklearn import datasets
np.random.seed()

Using TensorFlow backend.


In [29]:
df

,index,Unnamed: 0,band 1_20210201,band 2_20210201,band 3_20210201,band 4_20210201,band 5_20210201,band 6_20210201,band 7_20210201,band 1_20210305,...,band 6_20211226,band 7_20211226,band 1_20220119,band 2_20220119,band 3_20220119,band 4_20220119,band 5_20220119,band 6_20220119,band 7_20220119,label
0,0,0,41543,42574,41006,41195,36610,10033,11127,19851,...,7913,9267,24790,25681,24569,23662,13832,6013,7838,5
1,1,1,41799,42997,41175,41326,36739,10158,11237,16792,...,7986,9281,25666,27544,27159,24544,11566,6215,7862,5
2,2,2,41253,42181,40377,40714,36694,10118,11220,17992,...,7952,9302,27464,29365,29000,26330,14779,6579,8212,5
3,3,3,41045,41826,40260,40545,36641,10175,11209,20214,...,7792,9204,29573,31288,29557,28043,17391,6862,8408,5
4,4,4,41686,42655,40912,41313,38407,10496,11575,21517,...,7810,9200,31385,32733,31479,30345,20689,6894,8425,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98554,10946,7832,9959,8520,7378,4963,5544,7755,8440,10946,...,7610,8207,8425,7873,6389,5443,3803,4222,7321,4
98555,10947,7638,9458,7211,6241,3410,4240,7434,8231,10947,...,7089,7991,8209,7262,5575,4725,2807,3163,7049,4
98556,10948,7647,8367,6370,5784,2740,3862,7532,8363,10948,...,6985,8032,8202,6586,4845,4144,2040,2626,6975,4
98557,10949,7890,8265,6305,5905,2673,4049,7763,8555,10949,...,7046,8159,8410,7562,5888,5045,3048,3476,7324,4


In [25]:
def train(input, label, classes=7, epochs=100, batch_size=100, shuffle=True, model_path='model.h5'):
    assert classes>=2, 'number classese must be more than 1'
    model = Sequential()
    model.add(Dense(8, input_dim = 63))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(Dense(10, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(Dense(10, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(Dense(10, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(ReLU())

    # model.add(Dense(10, activation = 'relu'))
    # model.add(BatchNormalization())
    # model.add(ReLU())

    # model.add(Dense(10, activation = 'relu'))
    # model.add(BatchNormalization())
    # model.add(ReLU())
    if classes==2:
        model.add(Dense(2, activation = 'sigmoid'))
        loss = 'binary_crossentropy'
    else:
        model.add(Dense(classes, activation = 'softmax'))
        loss='categorical_crossentropy'

    model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])
    model.fit(input, label, epochs=epochs, batch_size=batch_size, shuffle=shuffle)
    scores = model.evaluate(input, label)
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    model.save(model_path)

In [2]:
def create_data_train(df):
    datasets = df.iloc[:, 2:]
    print(datasets.shape)
    X = datasets.iloc[:, :-1]
    Y = datasets.iloc[:, -1]
    encoder = LabelEncoder()
    encoder.fit(Y)
    encoded_Y = encoder.transform(Y)
    Y = np_utils.to_categorical(encoded_Y)
    return X,Y

In [31]:
fp_model_save = r"E:\WORK\Mongodia\Data_cut_img\modelDense.h5"
X_train, Y_train = create_data_train(df)
print(X_test.shape)
print('Training ...')
train(X_train, Y_train, classes=6, epochs=1000, batch_size=1000, shuffle=True, model_path=fp_model_save)





(98559, 64)
(19712, 63)
Training ...

Epoch 1/1000
98559/98559 [==============================] - 4s 40us/step - loss: 1.4287 - accuracy: 0.4430
Epoch 2/1000
98559/98559 [==============================] - 2s 23us/step - loss: 1.1566 - accuracy: 0.5751
Epoch 3/1000
98559/98559 [==============================] - 2s 24us/step - loss: 1.0012 - accuracy: 0.6485 0s - loss: 1.0046 - accuracy: 0.
Epoch 4/1000
98559/98559 [==============================] - 2s 20us/step - loss: 0.8057 - accuracy: 0.7398
Epoch 5/1000
98559/98559 [==============================] - 2s 20us/step - loss: 0.6338 - accuracy: 0.7986
Epoch 6/1000
98559/98559 [==============================] - 2s 20us/step - loss: 0.5090 - accuracy: 0.8365
Epoch 7/1000
98559/98559 [==============================] - 2s 20us/step - loss: 0.4365 - accuracy: 0.8568
Epoch 8/1000
98559/98559 [==============================] - ETA: 0s - loss: 0.3764 - accuracy: 0.88 - 2s 20us/step - loss: 0.3760 - accuracy: 0.8821
Epoch 9/1000
98559/98559 [=====

In [3]:
def create_predict_df_v2(list_fp_img, list_name_file_sort, list_number_band, csv_file):
    fp_img_first = [s for s in list_fp_img if list_name_file_sort[0] in s][0]
    df = get_df_flatten_predict(fp_img_first, list_number_band, list_name_file_sort[0])
    print(df.shape,"a")
    df.to_csv(csv_file)
    del df
    for name_file_sort in list_name_file_sort:
        fp_img = [s for s in list_fp_img if name_file_sort in s][0]
        df1 = get_df_flatten_predict(fp_img, list_number_band, name_file_sort)
        df = pd.read_csv(csv_file)
        df = pd.concat([df, df1], axis=1)
        df.to_csv(csv_file)
        print(df.shape)

In [4]:
dir_img = r"E:\WORK\Mongodia\Data\Img"
fp_csv = r"E:\WORK\Mongodia\a.csv"
# fp_model_save = r"E:\WORK\Mongodia\Data_cut_img\mode_num_round_100_max_depth7_gpu.model"
list_number_band = [1,2,3,4,5,6,7]

list_name_file_sort = get_list_image_by_time(dir_img)
list_fp_img = glob.glob(os.path.join(dir_img, "*.tif"))
# df_data = create_predict_df(list_fp_img, list_name_file_sort, list_number_band)
df_data = create_predict_df_v2(list_fp_img, list_name_file_sort, list_number_band, fp_csv)
# create_predict_df_v2
df_data = pd.read_csv(fp_csv)

NameError: name 'get_list_image_by_time' is not defined

In [5]:
from keras.models import load_model
from scipy.ndimage import convolve
src = rasterio.open(r"E:\WORK\Mongodia\Data_cut_img\label_shp\mask.tif")
profile = src.profile
fp_model_save = r"E:\WORK\Mongodia\Data_cut_img\modelDense.h5"
fp_predict = r"E:\WORK\Mongodia\timeserris\timeseris.tif"

para = 4
model = load_model(fp_model_save)
predictions = model.predict(df_data, batch_size=1000, verbose=1)
predictions = np.transpose(predictions)
predictions = np.reshape(predictions, (-1, src.height, src.width))
print('shape of predictions', predictions.shape)
kernel = [[1, 1, 1, 1, 1],
            [1, 1, 1, 1, 1],
            [1, 1, para, 1, 1],
            [1, 1, 1, 1, 1],
            [1, 1, 1, 1, 1]]
for i in range(len(predictions)):
    predictions[i] = convolve(predictions[i], kernel, mode='constant', cval=0.0)
test = np.argmax(predictions, axis=0).astype('uint8')
print(test.shape)
with rasterio.open(fp_predict, 'w', **profile) as dst:
    dst.write(np.array([test]))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



NameError: name 'df_data' is not defined

In [ ]:
# from keras.models import load_model
# from scipy.ndimage import convolve
# src = rasterio.open(r"E:\WORK\Mongodia\Data_cut_img\label_shp\mask.tif")
# profile = src.profile
# fp_model_save = r"E:\WORK\Mongodia\Data_cut_img\modelDense.h5"
# fp_predict = r"E:\WORK\Mongodia\timeserris\timeseris.tif"

# para = 4
# model = load_model(fp_model_save)
# predictions = []
# for i in range(df_data.shape[1]):
#     print('Pixel ', i)
#     prediction__ = model.predict(df_data[:,i,:], batch_size=1000, verbose=1)
#     if i==4: prediction__ = prediction__ * para
#     predictions.append(prediction__)
# predictions = np.sum(predictions, axis=0)
# """classfication"""
# test = np.argmax(predictions, axis=1)
# test=test.reshape(src.height, src.width)
# test=np.array([test.astype('uint8')])

# with rasterio.open(fp_predict, 'w', **profile) as dst:
#     dst.write(test)

In [35]:
predictions


array([[1.2911497e-17, 5.5467230e-13, 1.2042777e-05, 1.4435202e-05,
        5.3957218e-01, 4.6040127e-01],
       [1.2911497e-17, 5.5467230e-13, 1.2042777e-05, 1.4435202e-05,
        5.3957218e-01, 4.6040127e-01],
       [1.2911497e-17, 5.5467230e-13, 1.2042777e-05, 1.4435202e-05,
        5.3957218e-01, 4.6040127e-01],
       ...,
       [1.2911497e-17, 5.5467230e-13, 1.2042777e-05, 1.4435202e-05,
        5.3957218e-01, 4.6040127e-01],
       [1.2911497e-17, 5.5467230e-13, 1.2042777e-05, 1.4435202e-05,
        5.3957218e-01, 4.6040127e-01],
       [1.2911497e-17, 5.5467230e-13, 1.2042777e-05, 1.4435202e-05,
        5.3957218e-01, 4.6040127e-01]], dtype=float32)

In [6]:
import rasterio 
import numpy as np

src_mask = rasterio.open(r"E:\WORK\Mongodia\Data_cut_img\Label\label.tif")
mask = src_mask.read()
index = np.where(mask==0)


# viet lại 
path1 = r"X:\Linh\orkhon_mongolia\Ducanh\timeseris_dense.tif"
src_mask1 = rasterio.open(path1)
mask1 = src_mask1.read() + 1
mask1[index] = 0
mask1 = mask1.astype('uint8')
path1_return = r"X:\Linh\orkhon_mongolia\Ducanh\oke\timeseris_dense.tif"
profile = src.profile
profile.update(
        dtype=rasterio.uint8)
with rasterio.open(path1_return, 'w', **profile) as dst:
    dst.write(mask1)
    dst.write_colormap(
                1, {
                    1: (255, 0, 0, 255),
                    2: (255,255,0, 255),
                    3:(128,0,0,255),
                    4:(0,255,0,255),
                    5:(0,128,0,255),
                    6:(0,0,255,255) })

In [49]:
import rasterio 


src_mask = rasterio.open(r"E:\WORK\Mongodia\Data_cut_img\Label\label.tif")
mask = src_mask.read()
index = np.where(mask==0)


# viet lại 
path1 = r"X:\Linh\orkhon_mongolia\Ducanh\orkhon_2021_2022.tif"
src_mask1 = rasterio.open(path1)
mask1[index] = 0
mask1 = mask1.astype('uint8')
path1_return = r"X:\Linh\orkhon_mongolia\Ducanh\oke\orkhon_2021_2022.tif"
profile = src.profile
with rasterio.open(path1_return, 'w', **profile) as dst:
    dst.write(mask1)
    dst.write_colormap(
                1, {
                    1: (255, 0, 0, 255),
                    2: (255,255,0, 255),
                    3:(128,0,0,255),
                    4:(0,255,0,255),
                    5:(0,128,0,255),
                    6:(0,0,255,255) })

In [2]:
import rasterio 
import os, glob

dir_img = r"E:\WORK\Mongodia\Data\Img_Cut_box_predict_pixelbase_XGboost\tmp\Imgpredict_xgboostmodel_5000_v2_num_round_100_max_depth7_7Band_cut_img"
list_fp = glob.glob(os.path.join(dir_img, "*.tif"))

for path1 in list_fp:
    with rasterio.open(path1, 'r+') as dst:
        # dst.write(mask1)
        dst.write_colormap(
                    1, {
                        1: (255, 0, 0, 255),
                        2: (255,255,0, 255),
                        3:(128,0,0,255),
                        4:(0,255,0,255),
                        5:(0,128,0,255),
                        6:(0,0,255,255) })

In [10]:
from datetime import datetime
datetime.now().strftime('%Y_%m_%dwith%Hh%Mm%Ss')

'2022_04_19with23h10m19s'

In [1]:
import rasterio
import numpy as np

fp_img = r"E:\WORK\Mongodia\ThuDo_monggo\Data_training\predict\Dense_v2_model_Dense_add2Dense_2022_04_20with17h04m40s_label_mask_nobuildup.tif"
fp_buildUp = r"X:\Linh\stack_capital\Img_same_size\stress_8bit_perimage\stress\aaa.tif"

src_img = rasterio.open(fp_img)
src_buid = rasterio.open(fp_buildUp)

index_build = np.where(src_buid.read()==255)
mask = src_img.read() + 1
index_2 = np.where(mask == 2)
mask[index_2] = 1
mask[index_build] = 2
mask = mask - 1
with rasterio.open(fp_img, "r+") as dst:
    dst.write(mask)
    dst.write_colormap(
                    1, {
                        1: (237,2,42,255),
                        2: (255,219,92,255),
                        3:(167,210,130,255),
                        4:(200,200,200,255),
                        5:(238,207,168,255),
                        6:(53,130,33,255),
                        7:(26,91,171,255)
                         })
    


In [4]:
with rasterio.open(r"X:\Linh\stack_capital\Rs\Dense_v2_model_Dense_add2Dense_2022_04_20with17h04m40s_label_mask_nobuildup_mol.tif", "r+") as dst:
    # dst.write(mask)
    dst.write_colormap(
                    1, {
                        1: (237,2,42,255),
                        2: (255,219,92,255),
                        3:(167,210,130,255),
                        4:(200,200,200,255),
                        5:(238,207,168,255),
                        6:(53,130,33,255),
                        7:(26,91,171,255)
                         })
    